# CSV Format for Chatbot Flow
## Structure

The CSV should have the following columns:

1. **Question ID**
2. **Question**
3. **Option**: The options the user can choose.
4. **Response**: The chatbot's response based on the selected option.
5. **Next ID**: The ID of the next question to ask after the user selects this option.

## CSV layout


| Question ID | Question                                     | Option                            | Response                                                                          | Next ID |
|-------------|----------------------------------------------|-----------------------------------|----------------------------------------------------------------------------------|---------|
| 1           | Nice to meet you {Username}. How was your day? | Great                             | Wow, that is wonderful!                                                          | 2       |
| 1           | Nice to meet you {Username}. How was your day? | Good                              | Wow, that is wonderful!                                                          | 2       |
| 1           | Nice to meet you {Username}. How was your day? | Okay                              | An okay day is still progress. I hope that you found something good in it!      | 6       |
| 1           | Nice to meet you {Username}. How was your day? | Not good                         | I’m very sorry to hear that! Off days happen, but I’m here for you if you need anything. | 6       |
| 2           | What made today so good?                    | Quality time with others          | Ahh, that sounds 

## Notes

- Can add in the user's name using the `{Username}`
- To end the converstion leave the `Next ID` empty 

It will end with this text: 
- Okay, those are all the questions I have. Thanks for chatting!


In [5]:
import pandas as pd
import json

In [6]:
# Load the CSV file
df = pd.read_csv("chat.csv")

In [36]:

json_data = {}
#made with chat GPt
# Iterate through the DataFrame to build the JSON structure
for _, row in df.iterrows():
    question_id = str(row['Question ID'])
    question_text = row['Question']
    option_text = row['Option']
    response_text = row['Response']
    next_id = str(row['Next ID'])
    
    # Create the question entry if it doesn't exist
    if question_id not in json_data:
        json_data[question_id] = {
            "question": question_text,
            "options": {}
        }
    
    # Add the option to the question
    json_data[question_id]["options"][option_text] = {
        "response": response_text,
        "nextId": next_id
    }

# Convert the dictionary to JSON format
json_output = json.dumps(json_data, indent=4)

In [8]:

# Save the JSON output to a file
json_file_path = "chat.json"
with open(json_file_path, 'w') as json_file:
    json_file.write(json_output)

print("JSON file created successfully!")

JSON file created successfully!


# Calculating the time for each flow
(not including the time it takes for the user to make a choice)

In [23]:
with open('chat.json', 'r') as file:
    conversation_data = json.load(file)

In [24]:
# Typing speed in milliseconds per character
typing_speed = 50

In [30]:
#calculating the typing time for each question (and if there) response
def calculate_typing_time(question, response):
    time = len(question) * typing_speed  # Time for the question
    if response:  # Only add time for the response if it exists
        time += len(response) * typing_speed
    return time

In [31]:
#made with Chat GPT
def explore_flows(current_id, conversation_data, visited=None):
    """Recursively explore conversation flows and calculate total typing time."""
    if visited is None:
        visited = set()  # To track visited nodes

    # Base case: If this ID has been visited, return 0 to avoid cycles
    if current_id in visited:
        return 0

    visited.add(current_id)  # Mark this node as visited

    # Get the entry for the current ID, ensure it's a dict
    entry = conversation_data.get(current_id)
    if entry is None:
        return 0  # If the entry does not exist, return 0

    # Calculate time for the question and response
    question = entry.get("question", "")
    response = entry.get("response", "")
    total_time = calculate_typing_time(question, response)

    # Explore options if they exist and are a list
    options = entry.get("options")
    if isinstance(options, list):
        for option in options:
            next_id = option.get("next_id")
            if next_id is not None:
                total_time += explore_flows(next_id, conversation_data, visited)

    return total_time

In [34]:
#Made with chat gpt
def test_all_flows(conversation_data):
    """Test all conversation flows and return total typing time."""
    total_time = 0
    for starting_id in conversation_data:
        total_time += explore_flows(starting_id, conversation_data)
    return total_time

In [35]:

# Calculate the total typing time for all flows
total_typing_time = test_all_flows(conversation_data)
print(f"Total typing time for all flows: {total_typing_time} ms")
print(f"Total typing time for all flows: {total_typing_time / 1000} seconds")

Total typing time for all flows: 15100 ms
Total typing time for all flows: 15.1 seconds
